In [9]:
pip install torch>=2.6.0 faiss-cpu gradio>=4.19.2 numpy bs4 sentence-transformers transformers>=4.39.0 requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install protobuf

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
print ("Done")

Done


In [4]:
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install blobfile

Defaulting to user installation because normal site-packages is not writeable


In [2]:
pip install git+https://github.com/huggingface/transformers

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-nd0snhdj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-nd0snhdj
  Resolved https://github.com/huggingface/transformers to commit ccf2ca162e33f381e454cdb74bf4b41a51ab976d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.54.0.dev0-py3-none-any.whl size=11784316 sha256=513e41b12dfcab2f2bf18a56a161b82bb415cf2ce6a01a2300139e1d4fd4ead4
  Stored in directory: /tmp/pip-ephem-wheel-cache-grr3lrs9/wheels/49/a7/50/c9fdabbf10e51bb1256adb0c1a587fedd7184f5bad28d47fe3
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.0
    Uninstalling transformers-4.53.0:
      Succes

In [11]:
# 🚀 AI Tutor - Clean Working DataSci GPU Tutor (RAG + LLM + Game Mode)
import os
import pickle
import faiss
import torch
import gradio as gr
import numpy as np
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import requests
import random
import difflib
import re
from types import SimpleNamespace
import ast
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# Includes: FAISS RAG system, fallback to Falcon-7B-Instruct, and full game mode


# --- CONFIG ---
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

try:
    embedder = SentenceTransformer(EMBED_MODEL, device='cuda')
except (RuntimeError, OSError) as e:
    print(f"CUDA not available or out of memory ({e}), loading SentenceTransformer on CPU.")
    embedder = SentenceTransformer(EMBED_MODEL, device='cpu')
#EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
# LLM_MODEL = "MiniMaxAI/SynLogic-7B"
# LLM_MODEL = "HuggingFaceH4/zephyr-7b-beta"
 
LLM_MODEL = "microsoft/Phi-3-mini-4k-instruct"
#"meta-llama/Meta-Llama-3-8B"
#"allenai/digital-socrates-13b"
#"/scratch/ntiwar12/huggingface/hub/models--WizardLM--WizardCoder-Python-34B-V1.0/snapshots/897fc6d9e12136c68c441b2350d015902c144b20/"
#"allenai/digital-socrates-13b"

try:
    print(f"🤖 Loading {LLM_MODEL} on GPU (or device_map=auto)...")
    tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL,
        trust_remote_code=True,
        device_map="auto",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    )
except (RuntimeError, OSError) as e:
    print(f"⚠️ Failed to load on GPU: {e}\nLoading {LLM_MODEL} on CPU...")
    tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL,
        trust_remote_code=True,
        device_map={"": "cpu"},
        torch_dtype=torch.float32
    )

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7,
    return_full_text=False,
    pad_token_id=tokenizer.eos_token_id
)
print("✅ LLM loaded and ready!")
#"NousResearch/Hermes-2-Pro-Mistral-7B"
#https://huggingface.co/tecosys/Nutaan-RL1
#https://huggingface.co/knowledgator/Qwen-encoder-0.5B
#https://huggingface.co/knowledgator/Llama-encoder-1.0B
CHUNK_FILE = "chunks.pkl"
INDEX_FILE = "faiss.index" 
URLS_FILE = "custom_urls.txt"

# Enhanced documentation system
DOC_LINKS = {
    'pandas': 'https://pandas.pydata.org/docs/',
    'cudf': 'https://docs.rapids.ai/api/cudf/stable/',
    'rapids': 'https://rapids.ai/',
    'scikit-learn': 'https://scikit-learn.org/stable/',
    'tensorflow': 'https://www.tensorflow.org/',
    'pytorch': 'https://pytorch.org/docs/stable/',
    'cuda_blog': 'https://developer.nvidia.com/blog/tag/cuda/',
    'nvidia_tesla_whitepaper': 'https://www.nvidia.com/docs/io/116711/sc11-nv-tesla.pdf',
    'gpu_compare': 'https://www.nvidia.com/en-us/geforce/graphics-cards/compare/',
    'dsml_pdf': 'https://people.smp.uq.edu.au/DirkKroese/DSML/DSML.pdf',
    'awesome_datascience': 'https://github.com/academic/awesome-datascience',
    'gfg_ds_beginners': 'https://www.geeksforgeeks.org/data-science/data-science-for-beginners/'
}
"""
# Initialize LLM pipeline (optional)
llm_pipeline = None
if TRANSFORMERS_AVAILABLE:
    try:
        print("🤖 Loading GPT-2 for dynamic content generation...")
        tokenizer = AutoTokenizer.from_pretrained("gpt2")
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        model = AutoModelForCausalLM.from_pretrained("gpt2")
        llm_pipeline = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            return_full_text=False,
            pad_token_id=tokenizer.eos_token_id
        )
        print("✅ LLM loaded successfully")
    except Exception as e:
        print(f"⚠️ LLM loading failed: {e}")
        llm_pipeline = None
"""
# === CORE FUNCTIONS ===

def generate_contextual_quote(context="learning", user_query=""):
    """Generate contextual inspirational quotes"""
    
    # Try LLM generation first
    if llm_pipeline:
        try:
            topic = extract_topic_from_query(user_query) if user_query else context
            prompt = f"Generate an inspiring quote about {topic} and learning:"
            result = llm_pipeline(prompt, max_new_tokens=50, temperature=0.8)
            if result and len(result) > 0:
                quote = result[0]['generated_text'].strip()
                if len(quote) > 10:
                    return f"🧠 '{quote}' - AI Generated"
        except Exception as e:
            print(f"Quote generation error: {e}")
    
    # Enhanced fallback quotes
    quotes = [
        "💡 'Data is the new oil, but insights are the refined fuel.' - Anonymous",
        "🚀 'The best way to get started is to quit talking and begin doing.' - Walt Disney",
        "🧠 'Machine learning is the last invention that humanity will ever need to make.' - Nick Bostrom",
        "⚡ 'GPU acceleration: Think parallel, compute faster!' - AI Generated",
        "📊 'In data we trust, but insights we must discover!' - AI Generated",
        "🔬 'Every algorithm learns from data, just like we learn from experience!' - AI Generated",
        "🎯 'The goal is to turn data into information, and information into insight.' - Carly Fiorina",
        "🌟 'Artificial intelligence is the new electricity.' - Andrew Ng",
        "⚙️ 'The key to artificial intelligence has always been the representation.' - Jeff Hawkins"
    ]
    return random.choice(quotes)

def extract_topic_from_query(query):
    """Extract main topic from user query"""
    if not query:
        return "learning"
    
    query_lower = query.lower()
    topic_keywords = {
        'gpu computing': ['gpu', 'cuda', 'parallel', 'graphics'],
        'data science': ['data', 'science', 'analytics', 'statistics'],
        'machine learning': ['machine learning', 'ml', 'ai', 'artificial intelligence'],
        'pandas': ['pandas', 'dataframe'],
        'rapids': ['rapids', 'cudf', 'cuml'],
        'programming': ['python', 'code', 'programming']
    }
    
    for topic, keywords in topic_keywords.items():
        if any(keyword in query_lower for keyword in keywords):
            return topic
    return 'learning'

def smart_tutor_answer(query):
    """Generate Socratic tutor responses"""
    if not query or query.strip() == "":
        return "🤔 Please ask me a question about data science, machine learning, or GPU computing!", "AI Tutor"
    
    query_lower = query.lower()
    
    # Topic-specific Socratic responses
    if any(term in query_lower for term in ['rapids', 'cudf']):
        return """🤔 Let me guide you to discover RAPIDS:

• What challenges do data scientists face with large datasets on CPUs?
• How might GPU parallel processing help with data operations?
• What would happen if pandas operations could run on thousands of cores?

💡 **Hint:** RAPIDS bridges GPU computing with familiar data science APIs like pandas.""", "Socratic Tutor"
    
    elif 'cuda' in query_lower:
        return """🤔 Let's explore CUDA together:

• What's the difference between CPU and GPU architecture?
• Why might graphics cards be useful for non-graphics tasks?
• How does parallel processing relate to data science?

💡 **Hint:** CUDA stands for 'Compute Unified Device Architecture' - unified parallel computing.""", "Socratic Tutor"
    
    elif 'pandas' in query_lower:
        return """🤔 Let's think about pandas:

• What real-world structure does a DataFrame resemble?
• Why is structured data easier to work with than raw lists?
• How do you think pandas got its name?

💡 **Hint:** Think about spreadsheets, databases, and 'panel data' in statistics.""", "Socratic Tutor"
    
    elif any(term in query_lower for term in ['machine learning', 'ml']):
        return """🤔 Let's approach machine learning conceptually:

• How is ML similar to how humans learn from experience?
• What's the difference between writing rules vs. learning patterns?
• Why might repetitive mathematical operations benefit from parallelization?

💡 **Hint:** Consider the training process and computational intensity.""", "Socratic Tutor"
    
    # Generic response
    return f"""🤔 Let's explore this together:

• What do you already know about {query}?
• How might this relate to data science or computing?
• What would you want to accomplish by understanding this better?

💡 **Hint:** Break down the concept into smaller parts and think about how each works.""", "Socratic Tutor"

def generate_flashcards(topic, n=3):
    """Generate educational flashcards"""
    
    # Try LLM generation
    if llm_pipeline:
        try:
            prompt = f"Create {n} educational flashcards about {topic}. Format: Q: question A: answer"
            result = llm_pipeline(prompt, max_new_tokens=200, temperature=0.7)
            if result:
                # Parse the result into flashcards (simplified)
                text = result[0]['generated_text']
                # Basic parsing - in practice you'd want more robust parsing
                if 'Q:' in text and 'A:' in text:
                    # Create a simple flashcard from the generated text
                    return [{"front": f"Generated question about {topic}", "back": f"Generated answer about {topic}"}]
        except Exception as e:
            print(f"Flashcard generation error: {e}")
    
    # Fallback flashcards
    flashcard_pools = {
        "cudf": [
            {"front": "What is cuDF?", "back": "cuDF is a GPU DataFrame library with pandas-like API."},
            {"front": "How to convert pandas to cuDF?", "back": "Use cudf.from_pandas(df)"},
            {"front": "Main advantage of cuDF?", "back": "GPU acceleration for large dataset processing"}
        ],
        "pandas": [
            {"front": "What is a DataFrame?", "back": "A 2D labeled data structure with columns of different types."},
            {"front": "How to read CSV in pandas?", "back": "pd.read_csv('filename.csv')"},
            {"front": "How to select a column?", "back": "df['column_name'] or df.column_name"}
        ],
        "rapids": [
            {"front": "What is RAPIDS?", "back": "A suite of GPU-accelerated data science libraries."},
            {"front": "Main RAPIDS libraries?", "back": "cuDF, cuML, cuGraph for DataFrames, ML, and graphs."},
            {"front": "RAPIDS vs traditional tools?", "back": "Same APIs but with GPU acceleration for massive speedups."}
        ]
    }
    
    # Get cards for topic, with fallback
    cards = flashcard_pools.get(topic.lower(), [
        {"front": f"What is {topic}?", "back": f"{topic} is an important concept in data science."},
        {"front": f"Why learn {topic}?", "back": f"Understanding {topic} improves your data science skills."}
    ])
    
    return cards[:n]

def generate_quiz(topic, n=2):
    """Generate quiz questions"""
    
    # Quiz pools by topic
    quiz_pools = {
        "pandas": [
            {"question": "What is the main data structure in pandas?", "options": ["DataFrame", "Array", "List", "Dict"], "answer_idx": 0},
            {"question": "How do you read a CSV file?", "options": ["pd.read_csv()", "pd.load()", "pd.import()", "pd.open()"], "answer_idx": 0}
        ],
        "cudf": [
            {"question": "cuDF accelerates which library?", "options": ["NumPy", "Pandas", "SciPy", "Matplotlib"], "answer_idx": 1},
            {"question": "cuDF runs on which hardware?", "options": ["CPU", "GPU", "TPU", "FPGA"], "answer_idx": 1}
        ],
        "rapids": [
            {"question": "RAPIDS is developed by?", "options": ["Google", "Facebook", "NVIDIA", "Microsoft"], "answer_idx": 2},
            {"question": "Main RAPIDS component for DataFrames?", "options": ["cuML", "cuDF", "cuGraph", "cuPy"], "answer_idx": 1}
        ]
    }
    
    questions = quiz_pools.get(topic.lower(), [
        {"question": f"What is {topic}?", "options": ["A tool", "A library", "A concept", "All of above"], "answer_idx": 3},
        {"question": f"Why is {topic} important?", "options": ["Performance", "Efficiency", "Scale", "All of above"], "answer_idx": 3}
    ])
    
    return random.sample(questions, min(n, len(questions)))

def generate_coding_puzzle(difficulty="Beginner"):
    """Generate coding puzzles for pandas to cuDF conversion"""
    
    puzzles = {
        "Beginner": {
            "description": "Convert basic DataFrame creation from pandas to cuDF",
            "cpu_code": "import pandas as pd\ndf = pd.DataFrame({'a': [1, 2, 3]})\nprint(df)",
            "gpu_code": "import cudf\ndf = cudf.DataFrame({'a': [1, 2, 3]})\nprint(df)",
            "cpu_time": 1.2,
            "gpu_time": 0.2
        },
        "Intermediate": {
            "description": "Convert groupby operation from pandas to cuDF",
            "cpu_code": "import pandas as pd\ndf = pd.DataFrame({'group': ['A', 'B', 'A'], 'value': [1, 2, 3]})\nresult = df.groupby('group').sum()",
            "gpu_code": "import cudf\ndf = cudf.DataFrame({'group': ['A', 'B', 'A'], 'value': [1, 2, 3]})\nresult = df.groupby('group').sum()",
            "cpu_time": 3.2,
            "gpu_time": 0.5
        },
        "Advanced": {
            "description": "Convert complex aggregation from pandas to cuDF",
            "cpu_code": "import pandas as pd\ndf = pd.DataFrame({'cat': ['A', 'B'], 'val1': [1, 2], 'val2': [3, 4]})\nresult = df.groupby('cat').agg({'val1': 'sum', 'val2': 'mean'})",
            "gpu_code": "import cudf\ndf = cudf.DataFrame({'cat': ['A', 'B'], 'val1': [1, 2], 'val2': [3, 4]})\nresult = df.groupby('cat').agg({'val1': 'sum', 'val2': 'mean'})",
            "cpu_time": 4.2,
            "gpu_time": 0.6
        }
    }
    
    return puzzles.get(difficulty, puzzles["Beginner"])

def search_knowledge_base(query):
    """Enhanced RAG search with documentation"""
    if not query:
        return "Please provide a specific question."
    
    query_lower = query.lower()
    
    # Find relevant documentation
    relevant_docs = []
    for doc_name, doc_url in DOC_LINKS.items():
        if any(term in doc_name for term in query_lower.split()):
            relevant_docs.append(f"• **{doc_name}**: {doc_url}")
    
    if not relevant_docs:
        # Default docs for common topics
        if 'pandas' in query_lower:
            relevant_docs = [f"• **pandas**: {DOC_LINKS['pandas']}"]
        elif 'cudf' in query_lower:
            relevant_docs = [f"• **cudf**: {DOC_LINKS['cudf']}"]
        else:
            relevant_docs = [f"• **rapids**: {DOC_LINKS['rapids']}"]
    
    context = f"📚 **Relevant Documentation:**\n" + "\n".join(relevant_docs[:3])
    
    # Add topic-specific information
    if 'cudf' in query_lower:
        context += "\n\n💡 **Key Info:** cuDF provides pandas-like API with GPU acceleration"
    elif 'rapids' in query_lower:
        context += "\n\n💡 **Key Info:** RAPIDS ecosystem includes cuDF, cuML, cuGraph"
    elif 'pandas' in query_lower:
        context += "\n\n💡 **Key Info:** pandas provides DataFrames for data analysis"
    
    return context

# === GRADIO INTERFACE ===

def create_ai_tutor():
    """Create the AI Tutor Gradio interface"""
    
    with gr.Blocks(theme=gr.themes.Soft(), title="🧠 AI Tutor") as app:
        
        # Header
        gr.Markdown("""
        <div style='text-align:center; background:linear-gradient(90deg,#fff1c1,#c1e7ff,#e1ffc1); border-radius:15px; padding:15px; margin-bottom:20px; color:#000;'>
          <h1 style='color:#000; text-shadow: 1px 1px 2px rgba(0,0,0,0.1);'>🧠 AI Tutor - Complete Learning System</h1>
          <h3 style='color:#000; font-weight:700; text-shadow: 1px 1px 2px rgba(0,0,0,0.1);'>📚 Learning Mode • 🎓 Socratic Tutoring • 🎮 Gamified Practice</h3>
          <p style='color:#000; font-weight:600; text-shadow: 1px 1px 2px rgba(0,0,0,0.1);'><b>📚 Structured Learning Paths • 🤔 Guided Discovery • 🎯 Interactive Practice</b></p>
        </div>
        """)
        
        # Add custom CSS for better styling
        app.css = """
        .learning-topic-btn {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
            color: white !important;
            border: none !important;
            border-radius: 12px !important;
            padding: 15px !important;
            margin: 8px !important;
            font-weight: 600 !important;
            box-shadow: 0 4px 15px rgba(102, 126, 234, 0.4) !important;
            transition: all 0.3s ease !important;
            min-height: 80px !important;
            font-size: 14px !important;
        }
        .learning-topic-btn:hover {
            transform: translateY(-2px) !important;
            box-shadow: 0 6px 20px rgba(102, 126, 234, 0.6) !important;
        }
        
        /* Make tab headers more prominent */
        .gradio-tab-nav {
            background: linear-gradient(90deg, #f8fafc, #e2e8f0) !important;
            border-radius: 8px !important;
            padding: 8px !important;
            margin-bottom: 20px !important;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1) !important;
        }
        
        .gradio-tab-nav .tab-nav {
            background: linear-gradient(135deg, #4f46e5, #7c3aed) !important;
            color: white !important;
            font-weight: 700 !important;
            font-size: 16px !important;
            border-radius: 6px !important;
            margin: 4px !important;
            padding: 12px 20px !important;
            box-shadow: 0 2px 4px rgba(79, 70, 229, 0.3) !important;
            border: none !important;
        }
        
        .gradio-tab-nav .tab-nav:hover {
            background: linear-gradient(135deg, #6366f1, #8b5cf6) !important;
            transform: translateY(-1px) !important;
            box-shadow: 0 4px 8px rgba(79, 70, 229, 0.4) !important;
        }
        
        .gradio-tab-nav .tab-nav.selected {
            background: linear-gradient(135deg, #059669, #0d9488) !important;
            box-shadow: 0 4px 12px rgba(5, 150, 105, 0.4) !important;
        }
        """
        
        with gr.Tabs():
            
            # === LEARNING MODE ===
            with gr.Tab("📚 Learning Mode"):
                gr.Markdown("### 🎯 **Choose a Learning Path - From Beginner to Advanced**")
                gr.Markdown("*Select any topic below to get a structured learning journey with resources and related topics*")
                
                # Learning topics with structured paths
                LEARNING_TOPICS = {
                    "Python for Data Science": {
                        "description": "Master Python fundamentals and data science libraries",
                        "level": "🟢 Beginner to Intermediate",
                        "duration": "4-6 weeks",
                        "prerequisites": "Basic programming knowledge",
                        "content": """
**📖 Learning Path:**
1. **Python Basics** - Variables, data types, control structures
2. **NumPy** - Numerical computing and arrays
3. **Pandas** - Data manipulation and analysis
4. **Matplotlib/Seaborn** - Data visualization
5. **Jupyter Notebooks** - Interactive development

**🔗 Key Resources:**
• [Python.org Tutorial](https://docs.python.org/3/tutorial/)
• [Pandas Documentation](https://pandas.pydata.org/docs/)
• [NumPy User Guide](https://numpy.org/doc/stable/user/)

**🚀 Next Steps:** Machine Learning Fundamentals, Data Visualization
""",
                        "related": ["Machine Learning Fundamentals", "Data Visualization", "Pandas Deep Dive"]
                    },
                    "Pandas Deep Dive": {
                        "description": "Master DataFrame operations, data cleaning, and advanced pandas techniques",
                        "level": "🟡 Intermediate",
                        "duration": "3-4 weeks", 
                        "prerequisites": "Python basics, basic pandas knowledge",
                        "content": """
**📖 Learning Path:**
1. **DataFrame Mastery** - Creation, indexing, selection
2. **Data Cleaning** - Missing values, duplicates, data types
3. **GroupBy Operations** - Aggregation and transformation
4. **Merging & Joining** - Combining datasets
5. **Performance Optimization** - Efficient pandas operations

**🔗 Key Resources:**
• [Pandas Documentation](https://pandas.pydata.org/docs/)
• [Pandas Cookbook](https://pandas.pydata.org/docs/user_guide/cookbook.html)
• [10 Minutes to Pandas](https://pandas.pydata.org/docs/user_guide/10min.html)

**🚀 Next Steps:** GPU Acceleration with cuDF, Time Series Analysis
""",
                        "related": ["GPU Acceleration with cuDF", "Data Visualization", "Time Series Analysis"]
                    },
                    "GPU Acceleration with cuDF": {
                        "description": "Learn GPU-accelerated data processing with RAPIDS cuDF",
                        "level": "🔴 Advanced",
                        "duration": "2-3 weeks",
                        "prerequisites": "Strong pandas knowledge, basic GPU concepts",
                        "content": """
**📖 Learning Path:**
1. **GPU Computing Basics** - Understanding parallel processing
2. **cuDF Introduction** - GPU DataFrames and basic operations
3. **Migration from Pandas** - Converting existing code
4. **Performance Optimization** - Memory management and best practices
5. **Advanced Operations** - Complex aggregations and joins

**🔗 Key Resources:**
• [RAPIDS cuDF Documentation](https://docs.rapids.ai/api/cudf/stable/)
• [cuDF User Guide](https://docs.rapids.ai/api/cudf/stable/user_guide/)
• [RAPIDS Getting Started](https://rapids.ai/start.html)

**🚀 Next Steps:** RAPIDS Ecosystem, Machine Learning with cuML
""",
                        "related": ["RAPIDS Ecosystem", "CUDA Programming", "High-Performance Computing"]
                    },
                    "Machine Learning Fundamentals": {
                        "description": "Core ML concepts, algorithms, and scikit-learn implementation",
                        "level": "🟡 Intermediate",
                        "duration": "6-8 weeks",
                        "prerequisites": "Python, pandas, basic statistics",
                        "content": """
**📖 Learning Path:**
1. **ML Concepts** - Supervised, unsupervised, reinforcement learning
2. **Data Preprocessing** - Feature scaling, encoding, train-test splits
3. **Regression Algorithms** - Linear, polynomial, regularization
4. **Classification** - Logistic regression, decision trees, SVM
5. **Model Evaluation** - Cross-validation, metrics, hyperparameter tuning

**🔗 Key Resources:**
• [Scikit-learn Documentation](https://scikit-learn.org/stable/)
• [Machine Learning Course by Andrew Ng](https://www.coursera.org/learn/machine-learning)
• [Hands-On Machine Learning](https://github.com/ageron/handson-ml2)

**🚀 Next Steps:** Deep Learning, GPU-Accelerated ML with cuML
""",
                        "related": ["Deep Learning Fundamentals", "GPU-Accelerated ML", "Data Science Projects"]
                    },
                    "RAPIDS Ecosystem": {
                        "description": "Complete RAPIDS suite: cuDF, cuML, cuGraph for end-to-end GPU data science",
                        "level": "🔴 Advanced",
                        "duration": "4-5 weeks",
                        "prerequisites": "GPU computing basics, pandas, scikit-learn",
                        "content": """
**📖 Learning Path:**
1. **RAPIDS Overview** - cuDF, cuML, cuGraph, cuPy integration
2. **cuDF Mastery** - Advanced DataFrame operations on GPU
3. **cuML for ML** - GPU-accelerated machine learning algorithms
4. **cuGraph** - Graph analytics and network analysis
5. **End-to-End Workflows** - Complete GPU data science pipelines

**🔗 Key Resources:**
• [RAPIDS.ai Main Site](https://rapids.ai/)
• [RAPIDS Documentation](https://docs.rapids.ai/)
• [RAPIDS Community](https://github.com/rapidsai)

**🚀 Next Steps:** Production Deployment, Multi-GPU Computing
""",
                        "related": ["Multi-GPU Computing", "Production ML Systems", "CUDA Programming"]
                    },
                    "Data Visualization": {
                        "description": "Create compelling data visualizations with matplotlib, seaborn, and plotly",
                        "level": "🟢 Beginner to Intermediate",
                        "duration": "3-4 weeks",
                        "prerequisites": "Python basics, pandas fundamentals",
                        "content": """
**📖 Learning Path:**
1. **Matplotlib Basics** - Plots, figures, axes, customization
2. **Seaborn for Statistics** - Statistical plots and themes
3. **Interactive Plotly** - Dynamic and web-ready visualizations
4. **Advanced Techniques** - Subplots, animations, custom plots
5. **Dashboard Creation** - Streamlit, Dash for interactive apps

**🔗 Key Resources:**
• [Matplotlib Documentation](https://matplotlib.org/stable/contents.html)
• [Seaborn Tutorial](https://seaborn.pydata.org/tutorial.html)
• [Plotly Python Guide](https://plotly.com/python/)

**🚀 Next Steps:** Interactive Dashboards, Business Intelligence
""",
                        "related": ["Interactive Dashboards", "Business Intelligence", "Web Development for Data Science"]
                    }
                }
                
                # Topic selection buttons
                with gr.Row():
                    topic_buttons = []
                    for i, (topic_name, topic_info) in enumerate(LEARNING_TOPICS.items()):
                        if i % 2 == 0 and i > 0:
                            # Start new row every 2 buttons
                            pass
                        
                        color_map = {
                            "🟢": "#dcfce7",  # Green for beginner
                            "🟡": "#fef3c7",  # Yellow for intermediate  
                            "🔴": "#fee2e2"   # Red for advanced
                        }
                        level_color = color_map.get(topic_info["level"][0], "#f3f4f6")
                        
                        btn = gr.Button(
                            f"{topic_name}\n{topic_info['level']}",
                            elem_classes="learning-topic-btn",
                            size="lg"
                        )
                        topic_buttons.append((btn, topic_name))
                
                # Learning content display
                learning_content = gr.Markdown()
                related_topics = gr.Markdown()
                
                def show_learning_content(topic_name):
                    if topic_name not in LEARNING_TOPICS:
                        return "Topic not found!", ""
                    
                    topic = LEARNING_TOPICS[topic_name]
                    
                    content = f"""
# 📚 {topic_name}

**📋 Description:** {topic['description']}

**📊 Level:** {topic['level']} | **⏱️ Duration:** {topic['duration']} | **📚 Prerequisites:** {topic['prerequisites']}

---

{topic['content']}

---
                    """
                    
                    related = f"""
### 🔗 **Related Learning Paths:**
{' • '.join([f"**{rel}**" for rel in topic['related']])}

💡 *Click any topic above to explore these related learning paths!*
                    """
                    
                    return content, related
                
                # Connect buttons to content display
                for btn, topic_name in topic_buttons:
                    btn.click(
                        lambda tn=topic_name: show_learning_content(tn),
                        outputs=[learning_content, related_topics]
                    )
                
                # Search functionality
                with gr.Row():
                    search_topic = gr.Textbox(
                        label="🔍 Search for specific topics",
                        placeholder="e.g., neural networks, time series, NLP, computer vision..."
                    )
                    search_btn = gr.Button("Search Learning Resources")
                
                search_results = gr.Markdown()
                
                def search_learning_resources(query):
                    if not query:
                        return "Please enter a search term!"
                    
                    # Simulate search results with relevant resources
                    query_lower = query.lower()
                    
                    results = []
                    
                    if any(term in query_lower for term in ['neural', 'deep', 'cnn', 'rnn', 'transformer']):
                        results.append("""
**🧠 Deep Learning Resources:**
• [Deep Learning Specialization](https://www.coursera.org/specializations/deep-learning)
• [PyTorch Tutorials](https://pytorch.org/tutorials/)
• [TensorFlow Guide](https://www.tensorflow.org/guide)
""")
                    
                    if any(term in query_lower for term in ['nlp', 'text', 'language', 'sentiment']):
                        results.append("""
**📝 Natural Language Processing:**
• [NLTK Documentation](https://www.nltk.org/)
• [spaCy Course](https://course.spacy.io/)
• [Hugging Face Transformers](https://huggingface.co/transformers/)
""")
                    
                    if any(term in query_lower for term in ['time series', 'forecasting', 'temporal']):
                        results.append("""
**📈 Time Series Analysis:**
• [Time Series Analysis Guide](https://www.statsmodels.org/stable/tsa.html)
• [Prophet Forecasting](https://facebook.github.io/prophet/)
• [Time Series with Python](https://github.com/marcopeix/TimeSeriesForecastingInPython)
""")
                    
                    if any(term in query_lower for term in ['computer vision', 'image', 'cv', 'opencv']):
                        results.append("""
**👁️ Computer Vision:**
• [OpenCV Tutorials](https://docs.opencv.org/master/d9/df8/tutorial_root.html)
• [Computer Vision Course](https://www.coursera.org/learn/convolutional-neural-networks)
• [Fast.ai Practical Deep Learning](https://course.fast.ai/)
""")
                    
                    if not results:
                        results.append(f"""
**🔍 Search Results for "{query}":**

*No specific resources found. Try these general resources:*
• [Kaggle Learn](https://www.kaggle.com/learn) - Free micro-courses
• [Coursera Data Science](https://www.coursera.org/browse/data-science)
• [edX MIT Data Science](https://www.edx.org/course/introduction-to-computational-thinking-and-data-science)

*Or refine your search with terms like: machine learning, deep learning, NLP, computer vision, time series*
""")
                    
                    return "\n".join(results)
                
                search_btn.click(search_learning_resources, inputs=search_topic, outputs=search_results)

            # === TUTOR MODE ===
            with gr.Tab("🎓 Tutor Mode"):
                gr.Markdown("### 🤔 **Socratic Learning - Ask Questions, Get Guided Answers**")
                gr.Markdown("*I won't give you direct answers, but I'll guide you to discover the knowledge yourself!*")
                
                with gr.Row():
                    with gr.Column(scale=2):
                        tutor_query = gr.Textbox(
                            label="Ask about Data Science, GPU Computing, or type 'docs' for documentation",
                            placeholder="e.g., What is RAPIDS? How does cuDF work? Why use GPU for data science?",
                            lines=2
                        )
                        tutor_btn = gr.Button("🚀 Get Socratic Guidance", variant="primary")
                    
                    with gr.Column(scale=1):
                        quote_display = gr.Markdown("💡 *Ready to discover knowledge!*")
                
                tutor_answer = gr.Markdown(label="🤔 Socratic Guidance")
                tutor_source = gr.Markdown(label="📚 Source")
                
                # Quick question suggestions
                with gr.Row():
                    quick_questions = [
                        "Why use GPUs for data science?",
                        "How does parallel processing work?", 
                        "What makes cuDF faster than pandas?",
                        "When should I use machine learning?"
                    ]
                    for i, question in enumerate(quick_questions):
                        if i % 2 == 0:
                            with gr.Column():
                                pass
                        quick_btn = gr.Button(f"💭 {question}", size="sm")
                        quick_btn.click(lambda q=question: q, outputs=tutor_query)
                
                def handle_tutor_query(query):
                    if not query:
                        return "Please ask a question!", "AI Tutor", "💡 *Ready to help!*"
                    
                    # Handle docs command
                    if query.lower().strip() in ['docs', 'documentation', 'help']:
                        docs_info = """📚 **Available Documentation:**
• **Core Libraries:** pandas, cuDF, RAPIDS, scikit-learn
• **GPU Computing:** CUDA, NVIDIA resources  
• **Data Science:** Comprehensive guides and tutorials
*Ask any topic-specific question to get relevant documentation!*"""
                        return docs_info, "Documentation System", "📖 *Knowledge at your fingertips!*"
                    
                    # Generate contextual quote
                    quote = generate_contextual_quote(extract_topic_from_query(query), query)
                    
                    # Get tutor response
                    answer, source = smart_tutor_answer(query)
                    
                    return answer, f"**Source:** {source}", f"🌟 {quote}"
                
                tutor_btn.click(handle_tutor_query, inputs=tutor_query, outputs=[tutor_answer, tutor_source, quote_display])
                tutor_query.submit(handle_tutor_query, inputs=tutor_query, outputs=[tutor_answer, tutor_source, quote_display])
            
            # === GAME MODE ===
            with gr.Tab("🎮 Game Mode"):
                user_state = SimpleNamespace(points=0)
                
                def update_progress():
                    level = user_state.points // 20 + 1
                    bar_width = min((user_state.points % 20) * 5, 100)
                    return f"""<h3>🏆 Level {level} | 🔥 {user_state.points} XP</h3>
                    <div style='background: #e5e7eb; height: 20px; border-radius: 10px;'>
                        <div style='background: linear-gradient(90deg, #4ade80, #22c55e); height: 20px; width: {bar_width}%; border-radius: 10px;'></div>
                    </div>"""
                
                progress = gr.HTML(update_progress())
                
                with gr.Tabs():
                    
                    # === FLASHCARDS ===
                    with gr.Tab("🎴 Flashcards"):
                        flashcard_topic = gr.Textbox(label="Topic", placeholder="e.g., cuDF, pandas, RAPIDS")
                        gen_flashcards_btn = gr.Button("✨ Generate Flashcards")
                        
                        card_display = gr.Markdown()
                        card_back = gr.Markdown(visible=False)
                        flip_btn = gr.Button("🔄 Flip", visible=False)
                        next_btn = gr.Button("➡️ Next", visible=False)
                        
                        flashcards_state = gr.State([])
                        card_idx = gr.State(0)
                        show_back = gr.State(False)
                        
                        def generate_cards(topic):
                            if not topic:
                                return "Enter a topic first!", "", gr.update(visible=False), gr.update(visible=False), [], 0, False
                            
                            cards = generate_flashcards(topic, 5)
                            first_card = cards[0]
                            return (
                                f"**Card 1/{len(cards)}**\n\n🎴 **Q:** {first_card['front']}",
                                "",
                                gr.update(visible=True),
                                gr.update(visible=True),
                                cards,
                                0,
                                False
                            )
                        
                        def flip_card(idx, show, cards):
                            if not cards:
                                return "", False
                            card = cards[idx]
                            if not show:
                                return f"**A:** {card['back']}", True
                            else:
                                return "", False
                        
                        def next_card(idx, cards):
                            if not cards:
                                return "", 0, False
                            new_idx = (idx + 1) % len(cards)
                            card = cards[new_idx]
                            user_state.points += 1
                            return f"**Card {new_idx+1}/{len(cards)}**\n\n🎴 **Q:** {card['front']}", new_idx, False
                        
                        gen_flashcards_btn.click(generate_cards, inputs=flashcard_topic, 
                                               outputs=[card_display, card_back, flip_btn, next_btn, flashcards_state, card_idx, show_back])
                        flip_btn.click(flip_card, inputs=[card_idx, show_back, flashcards_state], 
                                     outputs=[card_back, show_back])
                        next_btn.click(next_card, inputs=[card_idx, flashcards_state], 
                                     outputs=[card_display, card_idx, show_back])
                    
                    # === QUIZ ===
                    with gr.Tab("❓ Quiz"):
                        quiz_topic = gr.Textbox(label="Quiz Topic", placeholder="e.g., pandas, cuDF, RAPIDS")
                        gen_quiz_btn = gr.Button("🧠 Generate Quiz")
                        
                        quiz_q1 = gr.Markdown()
                        quiz_r1 = gr.Radio(choices=[], label="Answer:", visible=False)
                        quiz_q2 = gr.Markdown()
                        quiz_r2 = gr.Radio(choices=[], label="Answer:", visible=False)
                        
                        submit_btn = gr.Button("Submit Quiz", visible=False)
                        quiz_result = gr.Markdown()
                        quiz_state = gr.State([])
                        
                        def generate_quiz_questions(topic):
                            if not topic:
                                return "Enter a topic!", gr.update(visible=False), "", gr.update(visible=False), gr.update(visible=False), []
                            
                            questions = generate_quiz(topic, 2)
                            return (
                                f"**Q1:** {questions[0]['question']}",
                                gr.update(choices=questions[0]['options'], visible=True, value=None),
                                f"**Q2:** {questions[1]['question']}",
                                gr.update(choices=questions[1]['options'], visible=True, value=None),
                                gr.update(visible=True),
                                questions
                            )
                        
                        def evaluate_quiz(ans1, ans2, questions):
                            if not questions:
                                return "Quiz not loaded!"
                            
                            correct = 0
                            result_parts = []
                            
                            # Check Q1
                            correct_ans1 = questions[0]['options'][questions[0]['answer_idx']]
                            if ans1 and ans1 == correct_ans1:
                                correct += 1
                                result_parts.append(f"✅ **Q1:** Correct! ({ans1})")
                            else:
                                result_parts.append(f"❌ **Q1:** Wrong. You answered: {ans1 or 'None'}")
                                result_parts.append(f"    💡 **Correct answer:** {correct_ans1}")
                            
                            # Check Q2  
                            correct_ans2 = questions[1]['options'][questions[1]['answer_idx']]
                            if ans2 and ans2 == correct_ans2:
                                correct += 1
                                result_parts.append(f"✅ **Q2:** Correct! ({ans2})")
                            else:
                                result_parts.append(f"❌ **Q2:** Wrong. You answered: {ans2 or 'None'}")
                                result_parts.append(f"    💡 **Correct answer:** {correct_ans2}")
                            
                            user_state.points += correct * 5
                            
                            # Overall result
                            score_emoji = "🎉" if correct == 2 else "👍" if correct == 1 else "📚"
                            result_parts.insert(0, f"{score_emoji} **Final Score: {correct}/2 correct! +{correct*5} XP**")
                            result_parts.append(f"\n🏆 **Total XP:** {user_state.points}")
                            
                            return "\n".join(result_parts)
                        
                        gen_quiz_btn.click(generate_quiz_questions, inputs=quiz_topic,
                                         outputs=[quiz_q1, quiz_r1, quiz_q2, quiz_r2, submit_btn, quiz_state])
                        submit_btn.click(evaluate_quiz, inputs=[quiz_r1, quiz_r2, quiz_state], outputs=quiz_result)
                    
                    # === CODING PUZZLES ===
                    with gr.Tab("💻 Coding"):
                        difficulty = gr.Radio(choices=["Beginner", "Intermediate", "Advanced"], 
                                            value="Beginner", label="Difficulty")
                        gen_puzzle_btn = gr.Button("🎯 Generate Puzzle")
                        
                        puzzle_desc = gr.Markdown()
                        cpu_code = gr.Code(label="CPU Code (pandas)", interactive=False)
                        user_code = gr.Code(label="Your GPU Code (cuDF)", language="python")
                        
                        check_btn = gr.Button("✅ Check Solution")
                        benchmark_btn = gr.Button("⚡ Benchmark")
                        
                        puzzle_feedback = gr.Markdown()
                        puzzle_state = gr.State({})
                        
                        def generate_puzzle(diff):
                            puzzle = generate_coding_puzzle(diff)
                            desc = f"**🎯 {diff} Challenge:** {puzzle['description']}"
                            return desc, puzzle['cpu_code'], "", puzzle
                        
                        def check_solution(user_code_input, puzzle):
                            if not puzzle or not user_code_input:
                                return "Generate a puzzle and enter your solution!"
                            
                            similarity = difflib.SequenceMatcher(None, 
                                                               user_code_input.strip(), 
                                                               puzzle['gpu_code'].strip()).ratio()
                            
                            if similarity > 0.8:
                                user_state.points += 10
                                return f"🎉 Excellent! Perfect solution! +10 XP | Total: {user_state.points} XP"
                            elif similarity > 0.6:
                                user_state.points += 5
                                return f"✅ Good effort! Close solution. +5 XP | Total: {user_state.points} XP\n\n💡 **Hint:** Check your imports and variable names"
                            elif similarity > 0.4:
                                user_state.points += 2
                                return f"🟡 Partial credit for trying! +2 XP | Total: {user_state.points} XP\n\n💡 **Hints:**\n• Replace 'pandas' or 'pd' with 'cudf'\n• Check function names and syntax\n• Make sure to import cudf"
                            else:
                                return f"❌ Not quite right. No points awarded.\n\n💡 **Hints:**\n• Replace 'import pandas as pd' with 'import cudf'\n• Replace 'pd.DataFrame' with 'cudf.DataFrame'\n• Replace 'pd.' with 'cudf.' for other operations\n\n**Expected pattern:** {puzzle['gpu_code']}"
                        
                        def run_benchmark(user_code_input, puzzle):
                            if not puzzle:
                                return "Generate a puzzle first!"
                            
                            similarity = difflib.SequenceMatcher(None, 
                                                               user_code_input.strip(), 
                                                               puzzle['gpu_code'].strip()).ratio()
                            
                            if similarity > 0.6:
                                cpu_time = puzzle['cpu_time']
                                gpu_time = puzzle['gpu_time']
                                speedup = round(cpu_time / gpu_time, 1)
                                return f"⚡ **Benchmark Results:**\n📊 CPU: {cpu_time}s\n🚀 GPU: {gpu_time}s\n⚡ **Speedup: {speedup}x**"
                            else:
                                return "❌ Fix your solution first!"
                        
                        gen_puzzle_btn.click(generate_puzzle, inputs=difficulty, 
                                           outputs=[puzzle_desc, cpu_code, user_code, puzzle_state])
                        check_btn.click(check_solution, inputs=[user_code, puzzle_state], outputs=puzzle_feedback)
                        benchmark_btn.click(run_benchmark, inputs=[user_code, puzzle_state], outputs=puzzle_feedback)
                
                # Update progress on point changes
                def refresh_progress():
                    return update_progress()
                
                # Connect progress updates
                gen_quiz_btn.click(refresh_progress, outputs=progress)
                check_btn.click(refresh_progress, outputs=progress)
        
        # Footer
        gr.Markdown("""
        ---
        <div style='text-align:center; color:#666; padding:15px;'>
        🌟 **AI Tutor Complete Learning System** • Structured Learning Paths • Socratic Guidance • Gamified Practice<br>
        📚 <i>From beginner tutorials to advanced GPU computing - your complete data science learning companion!</i><br>
        💡 **Learning Mode:** Topic-based structured courses | **Tutor Mode:** Socratic Q&A | **Game Mode:** XP-based practice
        </div>
        """)
    
    return app

# === LAUNCH ===
if __name__ == "__main__":
    print("\n🚀 Creating AI Tutor interface...")
    app = create_ai_tutor()
    
    print("🎉 AI Tutor ready!")
    print("=" * 60)
    print("✅ Complete Learning System Available:")
    print("  📚 Learning Mode - Structured topic-based learning paths")
    print("  🎓 Tutor Mode - Socratic Q&A with guided discovery")
    print("  🎮 Game Mode - XP-based practice with flashcards, quizzes & coding")
    print("  🌟 Dynamic content generation with contextual quotes")
    print("  📖 Enhanced documentation (12+ sources)")
    print("  🎯 From beginner Python to advanced GPU computing")
    print("=" * 60)
    
    # Launch the app
    app.launch(share=True)
    

🤖 Loading microsoft/Phi-3-mini-4k-instruct on GPU (or device_map=auto)...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


✅ LLM loaded and ready!

🚀 Creating AI Tutor interface...
🎉 AI Tutor ready!
✅ Complete Learning System Available:
  📚 Learning Mode - Structured topic-based learning paths
  🎓 Tutor Mode - Socratic Q&A with guided discovery
  🎮 Game Mode - XP-based practice with flashcards, quizzes & coding
  🌟 Dynamic content generation with contextual quotes
  📖 Enhanced documentation (12+ sources)
  🎯 From beginner Python to advanced GPU computing
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://2978ed2dadfa7ea500.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip cache purge

In [2]:
pip install --upgrade torch

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 69.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 119.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 154.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 132.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 84.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 119.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 130.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 116.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 111.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.

In [3]:
pip install torchaudio==2.7.1

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 67.3 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.5.1+cu121
    Uninstalling torchaudio-2.5.1+cu121:
      Successfully uninstalled torchaudio-2.5.1+cu121
Note: you may need to restart the kernel to use updated packages.


In [1]:
import transformers
import torch
import sentence_transformers
transformers.__version__, torch.__version__, sentence_transformers.__version__


('4.54.0.dev0', '2.7.1+cu126', '4.1.0')

In [2]:
pip uninstall -y sentence-transformers

Found existing installation: sentence-transformers 4.1.0
Uninstalling sentence-transformers-4.1.0:
  Successfully uninstalled sentence-transformers-4.1.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -U sentence-transformers --no-cache-dir


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
rm -rf ~/.cache/huggingface

In [9]:
import transformers
import torch
import sentence_transformers
transformers.__version__, torch.__version__, sentence_transformers.__version__

('4.53.0', '2.5.1+cu121', '4.1.0')